In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pprint import pprint
from torch.utils.data import DataLoader, TensorDataset


# Version 1

In [43]:
inputs = np.array([
    [73, 67, 43], 
    [91, 88, 64], 
    [87, 134, 58], 
    [102, 43, 37], 
    [69, 96, 70]
], dtype='float32')
targets = np.array([
    [56, 70], 
    [81, 101], 
    [119, 133], 
    [22, 37], 
    [103, 119]
], dtype='float32')

def ver1():
    np.random.seed(42)
    x = torch.from_numpy(inputs) # (None, 3)
    y = torch.from_numpy(targets) # (None, 2)
    
#     w = torch.randn(2, 3, requires_grad=True) # num_outputs, num_inputs
#     b = torch.randn(2, requires_grad=True)
    
    w = np.random.normal(scale=0.1, size=(2, 3))
    w = torch.tensor(w, dtype=torch.float32, requires_grad=True)
    b = np.random.normal(scale=0.1, size=(2))
    b = torch.tensor(b, dtype=torch.float32, requires_grad=True)
    
    pprint({'W': w, 'b': b})
    
    def model(x):
        return x @ w.t() + b # (None, 2)

    def mse(y_pred, y_true):
        diff = y_pred - y_true
        return torch.sum(diff * diff) / diff.numel()

    for i in range(2):
        y_pred = model(x)
        # print(y_pred)
        loss = mse(y, model(x)) 
        print(loss)
#         if i % 1000 == 0:
#             print(i, loss.item())
        loss.backward()
        
        pprint('dW')
        print(w.grad)
        
        with torch.no_grad():
            w -= w.grad * 1e-5
            b -= b.grad * 1e-5
            w.grad.zero_()
            b.grad.zero_()
    
            
ver1()

{'W': tensor([[ 0.0497, -0.0138,  0.0648],
        [ 0.1523, -0.0234, -0.0234]], requires_grad=True),
 'b': tensor([0.1579, 0.0767], requires_grad=True)}
tensor(7063.5811, grad_fn=<DivBackward0>)
'dW'
tensor([[-5663.6494, -6960.6177, -4136.5859],
        [-6730.3169, -8101.4106, -4872.5522]])
tensor(4947.7710, grad_fn=<DivBackward0>)
'dW'
tensor([[-4569.6934, -5779.8867, -3409.1182],
        [-5444.3818, -6714.4644, -4017.7886]])


# Version 2

In [28]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

def ver2():
    x = torch.from_numpy(inputs) # (None, 3)
    y = torch.from_numpy(targets) # (None, 2)
    train_ds = TensorDataset(x, y)
    # print(train_ds[0])
    train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)
    for i in train_dl:
        print(i)
        break
        
    model = nn.Linear(in_features=3, out_features=2)
    # print(model.weight, model.bias)
    loss_fn = nn.functional.mse_loss
    # opt = torch.optim.SGD(model.parameters(), lr=1e-5)
    opt = torch.optim.Adam(model.parameters())
    print(loss_fn(model(x), y))
    
    def fit(num_epochs, model, loss_fn, opt):
        for epoch in range(num_epochs):
            for xb, yb in train_dl:
                pred = model(xb)
                loss = loss_fn(pred, yb)
                loss.backward() # compute gradients
                opt.step() # update parameters using gradients
                opt.zero_grad()
            if epoch % 1000 == 0:
                print(epoch, loss.item())
                
    fit(10000, model, loss_fn, opt)
                
ver2()

[tensor([[102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.]]), tensor([[ 22.,  37.],
        [103., 119.],
        [ 22.,  37.],
        [ 81., 101.]])]
tensor(9130.4795, grad_fn=<MseLossBackward>)
0 6213.703125
1000 1.6922653913497925
2000 0.4158349931240082
3000 0.70238196849823
4000 0.5885873436927795
5000 0.528420627117157
6000 0.6098266243934631
7000 0.4385601580142975
8000 0.5441039204597473
9000 0.6226803660392761


In [196]:
model = nn.Linear(in_features=3, out_features=2)
print(model.weight, model.bias)
loss_fn = nn.functional.mse_loss

Parameter containing:
tensor([[ 0.3925, -0.4187, -0.3082],
        [ 0.5287, -0.1948, -0.2047]], requires_grad=True) Parameter containing:
tensor([-0.5586, -0.3306], requires_grad=True)
